In [ ]:
#https://en.wiktionary.org/wiki/Wiktionary:Main_Page
#Load damp  https://dumps.wikimedia.org/itwiktionary/20180301/itwiktionary-20180301-pages-meta-current.xml.bz2

# Read first 1000 Rows to get structure

In [1]:
file_content=[]
import bz2
with bz2.open("itwiktionary-20180301-pages-meta-current.xml.bz2",'r') as f:
    i  = 1000
    while (i >0):
        i  =i-1
        print(f.readline().decode('utf-8'))

<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.10/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.10/ http://www.mediawiki.org/xml/export-0.10.xsd" version="0.10" xml:lang="it">

  <siteinfo>

    <sitename>Wikizionario</sitename>

    <dbname>itwiktionary</dbname>

    <base>https://it.wiktionary.org/wiki/Pagina_principale</base>

    <generator>MediaWiki 1.31.0-wmf.22</generator>

    <case>case-sensitive</case>

    <namespaces>

      <namespace key="-2" case="case-sensitive">Media</namespace>

      <namespace key="-1" case="first-letter">Speciale</namespace>

      <namespace key="0" case="case-sensitive" />

      <namespace key="1" case="case-sensitive">Discussione</namespace>

      <namespace key="2" case="first-letter">Utente</namespace>

      <namespace key="3" case="first-letter">Discussioni utente</namespace>

      <namespace key="4" case="case-sensitive">Wikizionario</namespace>

      <namespac

# SAX parser to parse pages

In [5]:
import bz2
import xml.sax
class InkscapeSvgHandler(xml.sax.ContentHandler):

    def startDocument(self):
        self.parsed_data=[]
        
    def startElement(self, name, attrs):
        self.tag = name
        if name == "page":
            self.current_page={}
            
    def characters(self, content):
        if self.tag=='title':
            self.current_page['title']=content
            self.tag=''
        if self.tag=='text':
            if 'text' in self.current_page:
                self.current_page['text']= self.current_page['text']+content
            else:
                self.current_page['text']=content
        
    def endElement (self, name):
        if name == "page":
            self.parsed_data.append(self.current_page)
            
class HubErrorHandler(xml.sax.ErrorHandler):
    def __init__(self):
        pass
    def error(self, exception):
        import sys
        print ("Error, exception: %s\n" % exception)
    def fatalError(self, exception):
        print ("Fatal Error, exception: %s\n" % exception)

parser = xml.sax.make_parser()
content_parser= InkscapeSvgHandler()
parser.setContentHandler(content_parser)
parser.setErrorHandler(HubErrorHandler())
parser.parse(bz2.open("itwiktionary-20180301-pages-meta-current.xml.bz2","r"))

In [6]:
len(content_parser.parsed_data)

461468

# GET subset of parsed data to optimize performance

In [7]:
sub_set = content_parser.parsed_data[:100000]

In [8]:
del content_parser

In [76]:
import re
def extract_synonyms(text):
    synonyms=[]
    bloks = re.findall('{{-sin-}}.+\*{{Term', text,re.DOTALL)
    if len(bloks)>0:
        try:
            synonym_bloks = re.findall('\[\[.+\]\]', bloks[0])[0]
        except:
            #print(bloks[0])
            return []
        #synonyms.append(bloks[0])
        #print(synonym_bloks)
        synonyms=[synonym_blok[2:-2] for synonym_blok in synonym_bloks.split(', ')]
        #synonyms=[synonym for synonym in synonyms if len(synonym)==3]
    return synonyms


# Sample Extraction of 1 item

In [77]:
sub_set[1001]

{'text': "{{W|Dividere le traduzioni in base alla definizione a cui si riferiscono}}\n== {{-it-}} ==\n{{-sost-|it}}\n{{Pn}} ''m sing'' {{Linkp|accertamenti}}\n# l'accertare\n# {{Term|diritto|it}}{{Term|economia|it}}{{Term|medicina|it}} negozio con il quale si accerta la situazione [[giuridica]] o [[fiscale]] o [[sanitario|sanitaria]]\n#: ''accertamento d'imposta'', ''accertamenti sanitari''\n\n{{-sill-}}\n; ac | cer | ta | mén | to\n\n{{-pron-}}\n{{IPA|/atʧertaˈmento/}}\n\n{{-etim-}}\n{{Etim-link|accertare}}\n\n{{-sin-}}\n*[[verifica]], [[ricognizione]], [[riscontro]], [[controllo]], [[conferma]], [[constatazione]], [[riconoscimento]]\n*{{Term|diritto|it}} [[indagine]], [[investigazione]], [[ispezione]]\n*[[esame]], [[prova]], [[test]]\n\n{{-trad-}}\n{{trad1|}}\n:*{{el}}: [[διακρίβωση]] (diakrívosi) ''f''\n:*{{de}}: [[Ermittlung]] ''f'', [[Ausforschung]] ''f'', [[Feststellung]] ''f''\n{{trad2}}\n\n{{-ref-}}\n*{{Fonte|sin-co}}\n*{{Fonte|trec}}\n*{{Fonte|hoep}}\n*{{Fonte|gar}}\n*{{Fonte|

In [78]:
[(i['title'],extract_synonyms(i['text'])) for i in sub_set[1001:1002]]

[('accertamento',
  ['verifica',
   'ricognizione',
   'riscontro',
   'controllo',
   'conferma',
   'constatazione',
   'riconoscimento'])]

In [79]:
all_synonyms ={}
for i in sub_set:
    main_word = i['title']
    synonyms = extract_synonyms(i['text'])
    if len(synonyms)>0:
        all_synonyms[main_word]=synonyms

In [80]:
len(all_synonyms)

1703

# Results for top 100 pages

In [86]:
for key in list(all_synonyms.keys())[:100]:
    print('key: ', key)
    print('ynonyms: ', all_synonyms[key])    
    print('------------------------------')

key:  casa
ynonyms:  ['edificio', 'stabile', 'costruzione', 'fabbricato']
------------------------------
key:  libero
ynonyms:  ['sciolto', 'svincolato', 'affrancato', 'emancipato']
------------------------------
key:  parlare
ynonyms:  ['parlata', 'lingua', 'linguaggio', 'dialetto', 'idioma']
------------------------------
key:  azzurro
ynonyms:  ['turchese', 'celeste', 'turchino]],[[blu', 'pervinca']
------------------------------
key:  bianco
ynonyms:  ['candido]],[[netto', 'pulito', 'immacolato', 'puro', 'pallido', 'esangue', 'scolorito', 'canuto']
------------------------------
key:  giallo
ynonyms:  ['ocra', '[paglierino', 'canarino', 'zafferano']
------------------------------
key:  marrone
ynonyms:  ['castano', 'bruno']
------------------------------
key:  nero
ynonyms:  ['scuro', 'bruno', 'corvino', 'moro']
------------------------------
key:  rosa
ynonyms:  ['rosato']
------------------------------
key:  rosso
ynonyms:  ['scarlatto', 'vermiglio', 'purpureo', 'rossiccio', 'ama